 _Lambda School Data Science Unit 2_
 
 # Classification & Validation Sprint Challenge

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

#### For this Sprint Challenge, you'll predict whether a person's income exceeds $50k/yr, based on census data.

You can read more about the Adult Census Income dataset at the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/adult

#### Run this cell to load the data:

In [1]:
import pandas as pd

columns = ['age', 
           'workclass', 
           'fnlwgt', 
           'education', 
           'education-num', 
           'marital-status', 
           'occupation', 
           'relationship', 
           'race', 
           'sex', 
           'capital-gain', 
           'capital-loss', 
           'hours-per-week', 
           'native-country', 
           'income']

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', 
                 header=None, names=columns)

df['income'] = df['income'].str.strip()

## Part 1 — Begin with baselines

Split the data into an **X matrix** (all the features) and **y vector** (the target).

(You _don't_ need to split the data into train and test sets here. You'll be asked to do that at the _end_ of Part 1.)

In [8]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
y = df['income'] == '>50K'
X = df.drop(columns='income')

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You can answer this question either with a scikit-learn function or with a pandas function.)

In [12]:
y.value_counts(normalize=True)

False    0.75919
True     0.24081
Name: income, dtype: float64

What **ROC AUC score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of ROC AUC.)

In [13]:
from sklearn.metrics import roc_auc_score

majority_class = y.mode()[0]
y_pred = [majority_class] * len(y)

roc_auc_score(y, y_pred)

0.5

In this Sprint Challenge, you will use **"Cross-Validation with Independent Test Set"** for your model validaton method.

First, **split the data into `X_train, X_test, y_train, y_test`**. You can include 80% of the data in the train set, and hold out 20% for the test set.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)

## Part 2 — Modeling with Logistic Regression!

- You may do exploratory data analysis and visualization, but it is not required.
- You may **use all the features, or select any features** of your choice, as long as you select at least one numeric feature and one categorical feature.
- **Scale your numeric features**, using any scikit-learn [Scaler](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) of your choice.
- **Encode your categorical features**. You may use any encoding (One-Hot, Ordinal, etc) and any library (category_encoders, scikit-learn, pandas, etc) of your choice.
- You may choose to use a pipeline, but it is not required.
- Use a **Logistic Regression** model.
- Use scikit-learn's [**cross_val_score**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) function. For [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules), use **accuracy**.
- **Print your model's cross-validation accuracy score.**

In [19]:
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.impute import SimpleImputer

In [17]:
# Data exploration
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26048 entries, 5514 to 23654
Data columns (total 14 columns):
age               26048 non-null int64
workclass         26048 non-null object
fnlwgt            26048 non-null int64
education         26048 non-null object
education-num     26048 non-null int64
marital-status    26048 non-null object
occupation        26048 non-null object
relationship      26048 non-null object
race              26048 non-null object
sex               26048 non-null object
capital-gain      26048 non-null int64
capital-loss      26048 non-null int64
hours-per-week    26048 non-null int64
native-country    26048 non-null object
dtypes: int64(6), object(8)
memory usage: 3.0+ MB


In [27]:
# Create pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    MinMaxScaler(),
    LogisticRegression(solver='lbfgs', max_iter=1000)
)

# Calculate cross-validation score
score = cross_val_score(pipeline, X_train, y_train, cv=10, n_jobs=-1, scoring='accuracy').mean()
print('Cross-Validation Scores:', score)

Cross-Validation Scores: 0.8218675387931162


## Part 3 — Modeling with Tree Ensembles!

Part 3 is the same as Part 2, except this time, use a **Random Forest** or **Gradient Boosting** classifier. You may use scikit-learn, xgboost, or any other library. Then, print your model's cross-validation accuracy score.

In [29]:
from sklearn.ensemble import RandomForestClassifier

# Create pipeline
pipe = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    MinMaxScaler(),
    RandomForestClassifier(max_depth=None, n_estimators=100, n_jobs=-1, random_state=42)
)

score = cross_val_score(pipe, X_train, y_train, cv=10, n_jobs=-1, scoring='accuracy').mean()
print('Cross-Validation Score:', score)

Cross-Validation Score: 0.854384808842737


## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <td colspan="2" rowspan="2"></td>
    <td colspan="2">Predicted</td>
  </tr>
  <tr>
    <td>Negative</td>
    <td>Positive</td>
  </tr>
  <tr>
    <td rowspan="2">Actual</td>
    <td>Negative</td>
    <td style="border: solid">85</td>
    <td style="border: solid">58</td>
  </tr>
  <tr>
    <td>Positive</td>
    <td style="border: solid">8</td>
    <td style="border: solid"> 36</td>
  </tr>
</table>

In [32]:
true_positives = 36
false_positives = 58
true_negatives = 85
false_negatives = 8
actual_positives = true_positives + false_negatives
actual_negatives = true_negatives + false_positives
predicted_positive = true_positives + false_positives
predicted_negative = true_negatives + false_negatives

Calculate accuracy

In [33]:
accuracy = (true_positives + true_negatives) / (predicted_negative + predicted_positive)
print('Accuracy:', accuracy)

Accuracy: 0.6470588235294118


Calculate precision

In [34]:
precision = true_positives / predicted_positive
print('Precision:', precision)

Precision: 0.3829787234042553


Calculate recall

In [35]:
recall = true_positives / actual_positives
print('Recall:', recall)

Recall: 0.8181818181818182


## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Experiment with feature selection, preprocessing, categorical encoding, and hyperparameter optimization, to try improving your cross-validation score.

### Part 3
Which model had the best cross-validation score? Refit this model on the train set and do a final evaluation on the held out test set — what is the test score? 

### Part 4
Calculate F1 score and False Positive Rate. 